In [1]:
import boto3
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch import helpers
from requests_aws4auth import AWS4Auth
import os

In [2]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('yelp-restaurants')

response = table.scan()
restaurants = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    restaurants.extend(response['Items'])

In [3]:
print(len(restaurants))

5422


In [4]:
rest_tuple = []
for rest in restaurants:
    business_id = rest['business_id']
    cuisine = rest['cuisine']
    rest_tuple.append([business_id,cuisine])

In [5]:
print(len(rest_tuple))
print(rest_tuple[0])

5422
['XjeGryxde-tQZF_Ewu7NCw', 'Korean']


In [27]:
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
region = 'us-east-2'
service = 'es'

awsauth = AWS4Auth(AWS_ACCESS_KEY, AWS_SECRET_KEY, region, service)

host = 'search-restaurants-3o36qc56ks6wx2a727x7zhzxyi.us-east-2.es.amazonaws.com'
es = Elasticsearch(
    hosts = [{"host":host, 'port':443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
    )

for idx, rest in enumerate(rest_tuple):
    body={
        "business_id":rest[0],
        "cuisine": rest[1]}
    es.index(index='restaurants',doc_type='Restaurant',id=idx,body=body)

In [22]:
credentials = boto3.Session().get_credentials()
region = 'us-east-2'
service = 'es'
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


host = 'search-restaurants-3o36qc56ks6wx2a727x7zhzxyi.us-east-2.es.amazonaws.com'
es = Elasticsearch(hosts=[{"host": host, 'port': 443}],
                   http_auth = awsauth,
                   use_ssl=True,
                   verify_certs=True,
                   connection_class=RequestsHttpConnection)
res = es.search(index="restaurants",
                body={"query": {"match": {"cuisine": cuisine}}})

try:
    raw_result = res["hits"]["hits"]
    print(raw_result)
except KeyError:
    print("Error extracting hits from ES response")

all_res_ids = []
for res in raw_result:
    all_res_ids.append(res["_source"]["business_id"])

[{'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '311', '_score': 2.0279596, '_source': {'business_id': 'GCAQmtRdNhqVit5KGJ46Qg', 'cuisine': 'American'}}, {'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '88', '_score': 2.0279596, '_source': {'business_id': 'NavBPhyeykbve1uVSOrDPg', 'cuisine': 'American'}}, {'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '91', '_score': 2.0279596, '_source': {'business_id': '6ujYWjs1FTksvuVxuzjUYA', 'cuisine': 'American'}}, {'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '257', '_score': 2.0279596, '_source': {'business_id': 'aSbrPkclNZXBtA2k69Agsw', 'cuisine': 'American'}}, {'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '128', '_score': 2.0279596, '_source': {'business_id': '-SM_BQ5jiqvot_kl7RKoaQ', 'cuisine': 'American'}}, {'_index': 'yelp-restaurant', '_type': 'Restaurant', '_id': '338', '_score': 2.0279596, '_source': {'business_id': 'ECbvfMyJMd2SpNRb_01I5A', 'cuisine': 'American'}}, {'_in

In [26]:
es.indices.delete(index='restaurants')

{'acknowledged': True}